# Pasos finales  


## Eliminación de rayos cósmicos
mediante combinación de 3 o más imágenes similares (de existir), lo que precisa alineamiento previo, o limpieza individual mediante proceso semi-interactivo con el programa cleanest.

## Fotometría de las estrellas estándar

## Calibración fotométrica
Determinación de las constantes instrumentales y coeficientes de extinción para cada noche.


## Calibración astrométrica
Para ello se utilizará [Astrometry.net](http://astrometry.net/) y las herramientas disponibles en [AstrOmatic.net](15http://www.astromatic.net/).